### 창업계획서 초안 작성

사용자가 프롬프트 작성하면 기존 초안 작성 프롬프트에 맞춰서 작성해주고 챗지피티로 내용을 더 찾아서 적어준다.

In [ ]:
초안작성 llm 만들기
프롬프트를 작성해서 질문 문항에 대해 작성하게끔 
일단 여기까지만 해보자
프롬프트가 만들어지면 csv나 json에 페르소나를 만들어서 파일 넣고 작성이 되는지 확인 

In [5]:
from getpass import getpass
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain
# ConversationBufferMemory : 챗봇이 이전의 대화 내용을 기억할 수 있게 해주는 클래스
 # 사용하면 LLM에 질의를 넘겨줄 때 이전 대화도 같이 넘겨주게 됨
 # (단 이러한 memory클래스는 모델별로 최대 입력 토큰의 제약이 있어서 적절히 사용해야함)
from langchain.memory import ConversationBufferMemory
# ConversationBufferMemory : 챗봇이 이전의 대화 내용을 기억할 수 있게 해주는 클래스
 # 모든 대화 내용을 저장하지 않고 메모리와 토큰을 효율적으로 관리하고 싶을 때 사용
from langchain.memory import ConversationBufferWindowMemory  # 대화 몇 개까지 저장할꺼냐 지정할 수 있는 클래스

In [6]:
import json
import os
from openai import OpenAI

In [7]:
from getpass import getpass

In [8]:
MY_API_KEY = getpass("OpenAi API Key:")

In [9]:
client = OpenAI(api_key=MY_API_KEY)

In [10]:
from langchain.document_loaders import JSONLoader

### 기본 사업계획서 템플릿

In [41]:
아이템_명 = input('아이템 명을 입력하세요: ')
아이템_소개 = input('아이템 소개를 입력하세요: ')
예상_고객 = input('예상 고객을 입력하세요: ')
핵심_기술 = input('핵심 기술을 입력하세요: ')
시장_분석 = input('시장 분석을 입력하세요: ')
경쟁사_분석 = input('경쟁사 분석을 입력하세요: ')

# 입력된 내용을 딕셔너리에 저장
item_info = {
    '아이템 명': 아이템_명,
    '아이템 소개': 아이템_소개,
    '예상 고객': 예상_고객,
    '핵심 기술': 핵심_기술,
    '시장 분석': 시장_분석,
    '경쟁사 분석': 경쟁사_분석
}

# 결과 출력
print(item_info)

{'아이템 명': 'OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼', '아이템 소개': '처방전달시스템을 통해 나무병원, 조경업체, 케미컬사, 조경자재 판매사, 조경담당자가 수목진료 정보를 공유하여 수목관리 산업의 효율화 및 활성화를 도모하는 O2O 플랫폼임', '예상 고객': '전국적으로 1,850여개의 나무병원과 약 6,000여개의 조경업체가 목표 고객인 B2B 사업', '핵심 기술': '수목진료용 EMR 시스템을 통한 처방전달체계 네트워킹 구성', '시장 분석': '케미컬 판매업체 2000개와 나무병원 1800개, 그리고 6000여개의 수목시공 기업', '경쟁사 분석': '현재 경쟁사는 비숨이라는 업체'}


In [ ]:
base_template = {f'''
**아이템 명**  

**아이템 소개**  

**예상 고객**   

**핵심 기술**  

**시장 분석**  

**경쟁사 분석**  

**사업계획서 요약**

'''
}

In [ ]:
ai_prompt = {f'''
             사업 아이디어 구체화 : {item_info['아이템 명']}과 {item_info['아이템 소개']} 기반으로 {item_info['예상 고객']}
             }

In [45]:
ai_prompt = {f'''
             사업 아이디어 구체화 : 내 사업 아이디어를 구체화 해줘. 그리고 '아이템 명', '아이템 소개' 기반으로 '예상 고객' 부분을 작성해줘.
             시장 분석 : 사업에 필요한 시장분석을 도와줘. '아이템 소개' 기반으로 해당 아이템과 관련된 산업/제품의 '시장 분석' 부분을 작성해줘. 또한 트렌드와 '경쟁사 분석' 부분을 작성해줘.
             목표 설정 : 사업의 SMART 목표를 설정할 수 있도록 도와줘. [사업 분야]의 구체적이고 실현 가능한 목표를 제안해 줘.
             경쟁사 분석 : 내 경쟁사 분석을 도와줘. [경쟁사 이름]의 강점, 약점, 전략을 분석해 줘.
             '''
             }

In [ ]:
# 사업계획서 초안 생성을 위한 프롬프트 작성_예시

prompt = f"""
당신은 예비 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다. 내용을 풍족하게 써줄수록 합격 확률이 올라갑니다. 
자신의 작업에 자부심을 갖고 최선을 다해주세요.탁월함을 향한 헌식이 당신을 차별화합니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서 초안을 작성해주세요.
초안을 작성할 때는 {item_info}를 기반으로 쓰며 {ai_prompt}에 있는 작성 팁을 참고해서 작성해주세요.

초안 작성에 도움이 되는 양식 : {ai_prompt}
이용할 아이템에 대한 정보 : {item_info}
제공하는 템플릿 양식 : {base_template}

아이템 명: {item_info['아이템 명']}
아이템 소개: {item_info['아이템 소개']}
예상 고객: {item_info['예상 고객']}
핵심 기술: {item_info['핵심 기술']}
시장 분석: {item_info['시장 분석']}
경쟁사 분석: {item_info['경쟁사 분석']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'user','content': prompt}],
                                                        temperature=0.5)

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)


KeyError: '핵심 기술'

### 청년창업사관학교 전용 템플릿

In [11]:
아이템_명 = input('아이템 명을 입력하세요: ')
아이템_소개 = input('아이템 소개를 입력하세요: ')
제품_및_서비스_개발동기 = input('제품 및 서비스 개발동기를 입력하세요: ')
제품_및_서비스의_목적_필요성 = input('제품 및 서비스의 목적(필요성)을 입력하세요: ')
제품_및_서비스의_개발_방안 = input('제품 및 서비스의 개발 방안을 입력하세요: ')
자금_조달_및_개발_방안 = input('자금 조달 및 개발 방안을 입력하세요: ')
내수시장_확보_방안_경쟁_및_판매가능성 = input('내수시장 확보 방안(경쟁 및 판매가능성)을 입력하세요: ')
해외시장_진출_방안_경쟁_및_판매가능성 = input('해외시장 진출 방안(경쟁 및 판매가능성)을 입력하세요: ')
대표자_및_직원의_보유역량_및_기술보호_노력 = input('대표자 및 직원의 보유역량 및 기술보호 노력을 입력하세요: ')
사회적_가치_실천계획 = input('사회적 가치 실천계획을 입력하세요: ')
지역특화_아이디어_기반_사업추진계획 = input('지역특화 아이디어 기반 사업추진계획을 입력하세요: ')
예상_고객 = input('예상 고객: ')
시장_분석 = input('시장 분석을 입력하세요: ')
경쟁사_분석 = input('경쟁사 분석을 입력하세요: ')

# 입력된 내용을 딕셔너리에 저장
item_info = {
    '아이템 명': 아이템_명,
    '아이템 소개': 아이템_소개,
    '제품 및 서비스 개발동기': 제품_및_서비스_개발동기,
    '제품 및 서비스의 목적_필요성': 제품_및_서비스의_목적_필요성,
    '제품 및 서비스의 개발 방안': 제품_및_서비스의_개발_방안,
    '자금 조달 및 개발 방안': 자금_조달_및_개발_방안,
    '내수시장 확보 방안 경쟁 및 판매가능성': 내수시장_확보_방안_경쟁_및_판매가능성,
    '해외시장 진출 방안 경쟁 및 판매가능성': 해외시장_진출_방안_경쟁_및_판매가능성,
    '대표자 및 직원의 보유역량 및 기술보호 노력': 대표자_및_직원의_보유역량_및_기술보호_노력,
    '사회적 가치 실천계획': 사회적_가치_실천계획,
    '지역특화 아이디어 기반 사업추진계획': 지역특화_아이디어_기반_사업추진계획,
    '예상 고객': 예상_고객,
    '시장 분석': 시장_분석,
    '경쟁사 분석': 경쟁사_분석
}

# 결과 출력
print(item_info)


{'아이템 명': 'OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼', '아이템 소개': '처방전달시스템을 통해 나무병원, 조경업체, 케미컬사, 조경자재 판매사, 조경담당자가 수목진료 정보를 공유하여 수목관리 산업의 효율화 및 활성화를 도모하는 O2O 플랫폼임', '제품 및 서비스 개발동기': '나무의사의 진료기록을 작성, 보관하거나 처방 약품과 자재를 발주 및 관리하는 시스템이 전무해서', '제품 및 서비스의 목적_필요성': '나무병원에서 수목의료를 위해 사용하는 농약 및 비료, 자재 및 소모품 등의 재고 관리나 발주를 수작업으로 관리하고 있어 수목의료용 케미컬 및 자재 관리의 효율성이 떨어져서', '제품 및 서비스의 개발 방안': '나무의사용 EMR 시스템은 헬스케어 전문기업 시야인사이트 및 한양대학교 공과대학원과 함께 공동 개발하였으며, 향후 OCS 또한 함께 개발해 나갈 예정', '자금 조달 및 개발 방안': '시스템 백앤드 개발 일부를 외주맡길 예정이며 시스템 개발 및 서비스 관리 인력 2인을 채용할 계획', '내수시장 확보 방안 경쟁 및 판매가능성': '처방단계에서 농약에 대한 우선순위 제품 노출을 통해 농약사로부터 수익 창출. 농자재사로부터는 저마진 공급 계약을 통해 거래 수수료 발생', '해외시장 진출 방안 경쟁 및 판매가능성': '미국 산림청의 I-TREE 서비스 등 그린인프라와 관련한 산업이 확대되고 있으나 이를 시각화한 사례는 찾기 힘들다. 이에 병해충 확산 데이터 시각화 기술을 수출할 계획', '대표자 및 직원의 보유역량 및 기술보호 노력': "'대표는 한양대학교 도시대학원 랜드스케이프어바니즘 전공이며 인천시설공단과 자이에스엔디에서 10년 간 근무하였음.  기술 보호를 위해서는 특허 3건 출원", '사회적 가치 실천계획': '양질의 일자리로 장기간 함께 할 수 있는 체계를 구축할 예정으로 경영성과급을 도입할 계획임', '지역특화 아이디어 기반 사업추진계획': '', '예상 고객': '예상 고객으로는 내수시장 확보 방안에 나온 

In [1]:
kosme_template = {f'''

1. 문제 인식  
1-1. 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

1-2. 제품 및 서비스의 목적(필요성)
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)

2. 실현 가능성
2-1. 제품 및 서비스의 개발 방안
(제품(서비스) 구현정도ㅡ 제작 소요기간 및 제작방법(자체 외주), 추진일정 등을 기재)

3. 성장전략(Scale-up)
3-1. 자금소요 및 조달계획
(자금의 필요성, 금액의 적정성 여부를 판단할 수 있도록 사업비(정부지원금+대응자금)의 사용계획 등을 기재(신청사업의 통합관리지침 및 세부관리기준에 근거하여 작성))

3-2. 시장진입 및 성과창출 전략
3-2-1. 내수시장 확보 방안(경쟁 및 판매가능성) 
(내수시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재) 

3-2-2. 해외시장 진출 방안(경쟁 및 판매가능성)
(해외시장을 중심으로 주 소비자층, 주 타겟시장, 진출시기, 시장진출 및 판매 전략, 그간 실적 등을 구체적으로 기재)

4. 팀 구성
4-1. 대표자 및 직원의 보유역량 및 기술보호 노력

4-2. 사회적 가치 실천계획
(양질의 일자리 창출을 위한 중소기업 성과공유제, 비공유직의 정규직화, 근로시간 단축 등 사회적 가치 실천계획을 기재)

4-3. 지역특화 아이디어 기반 사업추진계획
(지역에 특화된 산업 및 문화 등을 신기술 또는 최신 트렌드와 결합한 측면에서 기술)


'''
}

In [2]:
kosme1_template = {f'''

1. 문제 인식  
1-1. 제품 및 서비스 개발동기
(자사가 개발(보유)하고 있는 제품 및 서비스에 대한 개발동기 등을 기재)

1-2. 제품 및 서비스의 목적_필요성
(제품(서비스)을 구현하고자 하는 목적, 고객의 니즈를 혁신적으로 해결하기 위한 방안 등을 기재)
'''
}

### Chain-of-Thought(논리적 사고) 기법 적용

In [3]:
kosme_pass = {f'''
사업계획서 해당 문항을 작성할 때 아래의 작성 TIP을 참고해서 작성해 주세요. 
아래 TIP은 정부지원사업 담당자가 알려준 내용이며, 참고할 경우 합격률을 대폭 늘릴 수 있습니다.

1. 해당 사업이 현재 많은 사람들이 겪는 문제점을 해결할 수 있다는 점을 어필합니다.
2. 그 문제점이 많은 사람이 겪는 불편함을 해결하고, 사회적 관점에서도 유익하다는 점을 어필합니다.
3. 최대한 구체적으로 작성해 주세요.    
4. 수치가 포함된 내용을 추가할 수 있다면 세부적인 수치도 입력해 주세요.
5. 입력한 시장의 트렌드, 시장 현황을 분석해 주세요.
6. 매출 발생 가능성과 사회 변화 가능성을 서술해 주세요.
7. 사업이 지속 가능적으로 성장하고 발전할지 적어주세요.
              
              '''}

In [4]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{ai_prompt}작성 팁을 참고하여 바로 사용이 가능한 사업계획서를 만들어줘.
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서 초안을 작성해주세요.

초안 작성에 도움이 되는 팁 : {ai_prompt}
이용할 아이템에 대한 정보 : {item_info}
제공하는 템플릿 양식 : {kosme_template}

아이템 명: {item_info['아이템 명']}
아이템 소개: {item_info['아이템 소개']}
문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}
성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}
팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}
예상 고객: {item_info['예상 고객']}
시장 분석: {item_info['시장 분석']}
경쟁사 분석: {item_info['경쟁사 분석']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7)

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)



KeyboardInterrupt



### 문제인식 : 제품 및 서비스 개발동기 

In [23]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=1)
                                                            
# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 1. 문제 인식

1-1. 제품 및 서비스 개발동기

나무병원을 운영하는 나무의사들은 수목 진료와 관련된 기록을 관리하는 체계적인 시스템이 부족하여 업무 효율성이 저하되고 있습니다. 나무의사의 진료기록이나 수목 처방 정보를 작성하고 보관할 수 있는 시스템이 존재하지 않아 정보의 누락이나 중복 발생이 빈번하게 일어나고 있습니다. 특히, 수작업으로 농약 및 자재를 발주하고 관리하는 과정에서 발생하는 비효율성은 업무의 전반적인 질을 떨어뜨리고 나무병원의 운영에 어려움을 더하고 있습니다. 이러한 문제를 해결하기 위해, OCS(처방전달시스템) 기반의 수목의료 관리 O2O 플랫폼을 개발하여 정보의 중앙 집중화를 도모하고, 이를 통해 수목 의료 산업의 효율성과 활성화를 증진하고자 합니다. 나무의사용 EMR시스템은 헬스케어 전문기업과의 협력을 통해 이미 구축하였으며, 앞으로 OCS 개발도 병행하여 수목진료 정보의 신속한 전달과 처방 과정을 최적화할 예정입니다.


### 문제인식 : 제품 및 서비스의 목적과 필요성

In [79]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 목적_필요성 부분만 작성해 주세요.

문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=1.0)
                                                            
# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 [제품 및 서비스의 목적 및 필요성]

도입할 OCS 기반의 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 수목관리 산업의 효율성을 크게 향상시키는 데 기여할 것입니다. 현재 나무병원에서 사용되는 농약, 비료 및 기타 자재와 소모품의 재고 관리 및 발주 프로세스는 주로 수작업에 의존하고 있어 비효율성이 존재합니다. 이러한 수작업 기반의 관리 방식은 인적 오류의 발생 가능성을 높이고, 재고의 정확성과 관리의 일관성을 확보하는데 어려움을 초래합니다. 따라서 이를 디지털화하고 체계적으로 관리할 수 있는 시스템 도입이 절실합니다.

이 플랫폼을 통한 디지털화는 나무병원 운영자들이 업무 효율성을 높이고, 실제 수목의료 용품의 사용 및 관리에서 생기는 문제점을 미연에 방지할 수 있는 기회를 제공합니다. 뿐만 아니라, 투명하고 체계적인 데이터 관리가 가능해져, 필요한 자원의 적시 공급과 효과적인 예산 관리가 가능하게 될 것입니다. 헬스케어 기술의 도입으로 정확한 진단과 처방이 가능하며, 이를 바탕으로 하는 맞춤형 케미컬 및 자재 추천으로 환자(수목)의 건강을 최우선으로 하는 관리 체계가 구현될 수 있습니다.

또한, 다양한 이해관계자들(나무병원, 조경업체, 케미컬사 및 자재 판매사 등) 간의 정보 공유 및 협업을 촉진하여, 생태계 전반의 경쟁력을 강화할 것입니다. 예를 들어, 조경담당자는 플랫폼을 통해 실시간으로 필요한 정보를 얻고, 이를 바탕으로 신속한 의사결정을 내릴 수 있습니다. 이는 결국 고객의 만족도와 신뢰도를 높이고, 수목 진료 서비스의 질을 높이는 데 기여할 것입니다.

이러한 필요성에 의해 플랫폼은 기존의 수작업 기반 시스템에서 벗어나 디지털 트랜스포메이션을 통해 산업 자체의 질적 성장과 발전을 이끄는 중요한 역할을 할 것입니다. 이를 통해 수목진료와 관리의 새로운 표준을 제시하며, 지속 가능한 그린인프라 구축에도 기여할 수 있습니다.


### 문장 생성의 다양성을 조장하는 4가지 생성 파라미터
1. Temperature : 쉽게말해 각 토큰이 선택될 확률을 조절하는 것. 낮을 수록 선택되는 토큰이 적어지고 높을 수록 다양한 토큰들이 선택되기에 다양한 답변을 받을 수 있다.
2. Top-K : 보수적인 대답을 원할 때 사용, 확률이 높은 k의 토큰만을 예측 대상 토큰으로 한정하여, 낮은 확률을 토큰이 선택될 확률을 낮춘다. 작아질수록 보수적 답변이며 1이면 항상 같은 답변
3. Top-P : 보수적인 대답을 원할 때 사용, 차이는 모르겠다.
4. Repetition_Penalty : 한 번 등장했던 토큰에 패널티를 부여하여 다음 예측에서 선택될 확률을 낮춰 문장이 더욱 다양한 단어로 구상될 수 있도록 함.(openAI에서 지원 안함)
5. frequency_penalty : 특정 토큰(단어 또는 구)이 생성된 횟수에 따라 페널티를 부여합니다. 즉, 동일한 단어가 반복될수록 해당 단어의 생성 확률을 감소시켜 중복을 방지합니다.(가독성 향상)
6. presence_penalty : 특정 토큰이 한 번이라도 생성되었을 경우, 해당 토큰의 생성 확률을 감소시켜 새로운 단어의 사용을 촉진합니다. 이를 통해 출력의 다양성을 높입니다.(아이디어 확장)

In [ ]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 \n\n1. 문제 인식  \n1-1. 제품 및 서비스 개발동기

나무병원과 조경업체에서 수목의료를 위해 사용하는 농약, 비료, 자재 등의 재고 관리와 발주 과정은 현재 대부분 수작업으로 이루어지고 있습니다. 이러한 전통적인 방식은 작업 효율성 저하뿐만 아니라 데이터 오류 발생 가능성을 높입니다. 특히 나무의사의 진료 기록을 작성하고 보관하는 체계가 부족하여 진료 정보의 지속적인 추적과 관리를 힘들게 합니다.

이러한 문제는 단순히 업무 효율성에 그치지 않고, 나무 건강 상태의 적절한 평가와 치료에도 영향을 미칠 수 있습니다. 예를 들어, 과거 진료 기록이나 처방 내역이 부실하면 특정 병충해에 대한 정확한 대응이 어렵습니다. 이는 결과적으로 나무 건강 악화 및 생태계 균형 파괴로 이어질 수 있으며, 이에 따라 조경 산업 전체에도 부정적인 영향을 미칩니다.

또한 케미컬사 및 자재 판매사는 관련 데이터를 실시간으로 공유받지 못하여 공급망 관리에서 어려움을 겪습니다. 이로 인해 불필요한 재고 비용 증가 및 납품 지연 등이 발생할 수 있어 시장 경쟁력 약화를 초래합니다.

따라서 OCS 기반의 수목 의료용 케미컬 및 자재 커머스 플랫폼을 통해 이러한 문제를 해결하고자 합니다. 시야인사이트 등 전문 기업과 협력하여 EMR 시스템을 공동 개발함으로써 효과적인 데이터 관리와 공유가 가능하도록 하였습니다.

이를 통해 나무병원부터 관련 업체까지 모든 이해 관계자가 참여하는 통합된 플랫폼 환경을 구축함으로써 효율성을 극대화하고 산업 활성화를 도모하려는 것이 본 사업의 개발 동기입니다.



### presence_penalty만 사용

In [28]:
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=1,
                                                            presence_penalty=1)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 1. 문제 인식  
1-1. 제품 및 서비스 개발동기  
현재 수목의료산업에서는 나무의사의 진료기록을 체계적으로 작성하고 보관할 수 있는 시스템이 부족한 상황입니다. 이는 진료 이력을 추적하고 분석하는 데 있어 큰 불편을 초래하며, 진료의 일관성과 정확성을 떨어뜨리는 요인이 됩니다. 또한, 수목 의료에 필요한 약품과 자재의 발주 및 관리를 수작업으로 진행하고 있어 효율성이 낮고 오류가 발생하기 쉽습니다. 이러한 문제점들은 나무병원과 조경업체들이 효과적으로 운영되는 데 있어 중요한 장애물로 작용하고 있습니다. 따라서, OCS 기반의 통합 플랫폼을 통해 나무의사와 관련 업체들이 편리하고 체계적으로 정보를 관리 할 수 있게 하여 수목관리 산업의 효율성을 높이는 것이 필수적입니다.


### frequency_penalty만 사용

In [29]:
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=1,
                                                            frequency_penalty=1)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 1. 문제 인식  
1-1. 제품 및 서비스 개발동기  
나무의사의 역할은 점점 더 중요해지고 있습니다. 특히 도시화가 진행되며 생태계를 보전하고 녹지 공간을 관리하는 데 있어 나무의사의 전문성이 강조되고 있습니다. 그러나 현재, 나무의사는 진료 기록 작성, 보관 또는 처방 약품과 자재를 발주 및 관리할 수 있는 체계적 시스템이 부족한 상황입니다. 이러한 시스템 부재는 효율적인 수목 관리에 큰 장애물로 작용하며, 결과적으로 산업 전체의 비효율성을 초래합니다. 이로 인해 우리는 OCS 기반 수목 의료용 케미컬 및 자재 커머스 O2O 플랫폼을 개발하게 되었습니다. 이는 나무병원과 관련 업체들이 보다 효과적으로 정보를 공유하고 업무를 수행할 수 있도록 도와줍니다.

나무병원에서는 필요 시 농약이나 비료, 기타 소모성 자재들을 발주해야 하지만 대부분 기존 방식으로 개별 관리하고 발주를 진행함으로써 재고 관리에 어려움을 겪고 있습니다. 이러한 문제는 시간과 비용 낭비로 이어지며 결과적으로 진료의 품질에도 영향을 미칩니다. 우리의 플랫폼은 이러한 과정들을 디지털화하고 자동화하여 개선점을 제시하려 합니다.

또한 급증하는 민간 소비자들의 자연환경 관심 증가와 유지관리 요구에 부응하기 위해서는 일선에서 활동하는 전문가들의 데이터 기반 관리를 통한 병충해 예방 및 치료가 중요한 변수로 작용합니다 정밀한 데이터 입력 작업 과정을 간소화하여 업무 집중도를 높이고 필요한 정보를 빠르게 제공함으로써 정확성과 효율성을 증대시키기 위한 목적으로 개발되었습니다.

마지막으로 기존 산림청이나 조경관련 공공기관에서 자체 구축된 시스템 대비 통합적 기능을 갖춘 사업 모델을 제공함으로써 공공 뿐만 아니라 민간시장 확대 가능성 또한 열리고 있다는 점입니다 지역 특성 맞춤형 정보제공도 가능하게 하여 혁신적인 정보 융합 솔루션 제공합니다.

전반적으로, 본 프로젝트는 단순히 기술개발 그 이상의 의미를 지니고 있으며 미래 환경 변화 대응력 강화를 목표로 하는 창조적 노력임을 확신합니

### 2. 목적과 필요성 프롬프트

In [78]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
항목당 5줄 이상을 적어주세요.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 목적_필요성 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7,
                                                            presence_penalty=0.8,
                                                            frequency_penalty=0.5)
                                                            
# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 **제품 및 서비스의 목적_필요성**

OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 수목관리 산업에서 발생하는 비효율적인 재고 관리와 발주 과정을 혁신적으로 개선하기 위해 개발되었습니다. 현재 나무병원에서는 농약, 비료, 자재 및 소모품 등을 수작업으로 관리하고 발주를 처리하고 있습니다. 이는 시간과 인력의 낭비를 초래하며, 잘못된 주문이나 재고 부족 등의 문제로 인해 수목 치료에 지장을 줄 수 있습니다.

이 플랫폼은 이러한 문제를 해결하여 수목의료 현장에서 효율성과 정확성을 높이고자 합니다. 나무의사의 진료기록을 디지털화하고 처방 약품과 자재를 시스템 내에서 직접 발주할 수 있도록 함으로써, 사용자는 더 이상 종이 문서나 엑셀 파일을 사용할 필요가 없게 됩니다. 또한, 플랫폼은 실시간 재고 상태와 예상 소모량을 분석하여 최적의 발주 시점을 제안함으로써 과잉 재고나 품절 사태를 방지합니다.

더불어, 이 플랫폼은 농약사와 자재 공급업체와의 연결을 강화하여 제품군 추천 기능을 통해 사용자에게 가장 적합한 솔루션을 제공합니다. 이를 통해 사용자는 신뢰할 수 있는 정보를 바탕으로 보다 빠른 의사 결정을 할 수 있으며, 제품 선택에 대한 만족도를 높일 수 있습니다. 따라서 본 서비스는 고객의 니즈를 중심으로 한 맞춤형 솔루션 제공이라는 측면에서 차별화된 가치를 제시합니다.

결론적으로 OCS 기반 플랫폼은 단순히 업무 프로세스를 자동화하는 것을 넘어 전체적인 산업 구조 변화를 도모합니다. 이를 통해 업계 전반에 걸쳐 표준화된 데이터 접근성을 제공하고 정보 공유를 활성화함으로써 지속 가능한 성장 가능성을 열어갑니다. 이 과정에서 사회적 관점에서도 환경 보호 및 그린 인프라 발전에 기여하는 긍정적인 효과가 기대됩니다.


### 초안 3개 작성 후 점수를 부여하여 가장 높은 점수의 초안 출력

In [ ]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme1_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
항목당 5줄 이상을 적어줘.
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
"""

# OpenAI API를 사용하여 사업계획서 초안 3개 생성
drafts = []
for _ in range(3):
    completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7, # 창의성과 논리성의 균형(비즈니스 분석과 논리적 사고, 전략젝 제안이 필요하여)
                                                            presence_penalty=0.8, # 기존 내용에서 벗어난 새로운 개선점 추가를 위해
                                                            frequency_penalty=0.5) # 다양한 표현을 주되, 중요 개념의 반복도 가능
    drafts.append(completion.choices[0].message.content)

# 생성된 사업계획서 초안 출력
for i, draft in enumerate(drafts, start=1):
    print(f"{i}번 초안:\n", draft)

1번 초안:
 ### 1. 문제 인식

#### 1-1. 제품 및 서비스 개발동기

현대 사회에서 도시 녹지와 수목의 관리 필요성이 증가하고 있습니다. 하지만 나무병원과 조경업체들은 여전히 전통적인 방식에 의존하여 진료 기록을 작성하고, 처방 약품과 자재를 발주 및 관리하고 있습니다. 이는 관리의 비효율성을 초래하며, 수목 의료의 질적 저하로 이어질 가능성이 높습니다.

특히, 현재 나무의사의 진료 기록은 대부분 수작업으로 이루어져 있어 데이터의 체계적인 보관 및 활용에 한계가 있습니다. 이러한 비효율적인 시스템은 신속한 정보 공유와 협력을 방해하며, 결과적으로 수목 건강 관리에서 중요한 의사결정 과정을 지연시킵니다.

수목관리 산업 내에서 케미컬 및 자재 사용에 대한 정확한 정보 전달이 부족하여, 잘못된 제품 사용으로 인한 추가 손해나 환경 문제 발생 위험도 존재합니다. 따라서 효율적이고 체계적인 정보 관리 시스템 필요성이 대두되고 있으며, 이는 많은 수목관리 종사자들이 경험하는 공통된 문제입니다.

본 플랫폼은 이러한 문제를 해결하기 위해 OCS 기반의 수목 의료 전문 O2O 플랫폼을 제안합니다. 이를 통해 나무병원, 조경업체 등 관련 기관들이 더 효과적으로 정보를 공유하고 협력함으로써 산업 전반의 효율성을 크게 향상시킬 수 있을 것으로 기대됩니다.

더불어 본 플랫폼은 단순한 업무 효율성 개선을 넘어 관련 종사자들의 작업 환경 개선과 새로운 일자리 창출에도 기여할 것입니다. 지속 가능한 사업 모델을 통해 장기적으로 성장할 가능성도 매우 높다고 판단됩니다.

2번 초안:
 1. 문제 인식  

1-1. 제품 및 서비스 개발동기

현대 도시와 농업 환경에서 수목의료는 필수적인 부분으로 자리 잡고 있으며, 나무의사들은 수목의 건강을 관리하고 보호하는 중요한 역할을 맡고 있습니다. 그러나 현재 나무의사의 진료기록 작성, 보관 및 처방 약품과 자재를 발주하고 관리하는 시스템이 거의 전무한 상태입니다. 이로 인해 수목관리 과정에서 많은 비효율성과 오류가 발생하며, 

In [72]:
# 개발동기 출력물.
import json

def evaluate_draft(draft):
    """
    GPT-4o를 활용해 초안 평가 및 개선점 분석
    KOSME 평가 기준 반영: 문제 정의의 구체성과 설득력(40%), 차별화 요소 및 신뢰성 확보(30%), 시장 확장 가능성(20%), 논리적 전개(10%)
    """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성개 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "strengths": ["평가항목1 관련 강점", "평가항목2 관련 강점"],
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}

def select_best_draft(drafts):
    """초안 평가 및 최적 선택"""
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 선택된 초안의 핵심 강점(심사기준 연계)
    2. 잠재적 취약점 및 리스크 
    3. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }
    
    
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft']) 

★ 최종 선택: 2번 초안 (83/100)

▼ 합격률 높은 이유 분석 ▼
**분석 리포트**

1. **선택된 초안의 핵심 강점(심사기준 연계)**

   - **문제 정의의 구체성과 설득력:** 초안은 나무의사의 진료기록 작성 및 재고 관리 시스템 부재로 인한 비효율성 문제를 명확히 설명하고 있습니다. 이러한 문제는 수목 건강에 부정적인 영향을 미칠 수 있다는 점에서, 문제의 심각성을 효과적으로 전달하고 있습니다.
   
   - **차별화 요소 및 신뢰성 확보:** OCS 기반 플랫폼 도입을 통해 정보 공유와 협력을 강화하려는 접근 방식이 독창적이며 실현 가능성이 높습니다. 이는 기존 수작업 방식의 한계를 극복할 수 있는 현실적이고 혁신적인 해결책으로 평가됩니다.

2. **잠재적 취약점 및 리스크**
   
   - **시장 확장 가능성 부족:** 현재 초안은 특정 시장(수목관리)에만 초점을 맞추고 있어, 다양한 응용 분야에 대한 고려가 부족합니다. 이는 잠재적으로 시장 확대 기회를 놓치게 할 수 있는 위험 요소입니다.
   
   - **논리적 전개 부족:** 프로젝트의 환경 지속 가능성과 일자리 창출 효과에 대한 구체적인 데이터나 사례가 부족하여 논리적인 전개가 약해질 수 있습니다. 이는 제안서의 설득력을 저하시킬 위험이 있습니다.

3. **KOSME 가이드라인 반영한 구체적 개선방안**
   
   - **구체적 개선방안 1:** 플랫폼의 기능과 장점을 다른 관련 산업에도 확장할 수 있는 방안을 제시하여 시장 확장 가능성을 보강해야 합니다. 예를 들어, 도시 녹지 관리, 공원 관리 등 다양한 응용 분야에서 해당 플랫폼을 활용할 수 있는 방법을 연구하고 제시해야 합니다.
   
   - **구체적 개선방안 2:** 환경 지속 가능성과 일자리 창출 효과를 입증하기 위한 구체적인 데이터나 사례 연구를 포함하여 논리적 전개를 강화해야 합니다. 이를 위해 유사한 기술이나 플랫폼 도입 사례 분석, 예상되는 경제적/사회적 파급 효과에 대한 정량적 데이터를 제

### 항목 2개 한 번에 뽑기

In [13]:
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
"""

# OpenAI API를 사용하여 사업계획서 초안 생성
completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7,
                                                            presence_penalty=-0.8,
                                                            frequency_penalty=0.5,
                                                            max_tokens = 4000)
                                                            
# presence_penalty: 이 값을 0.5~1.0 정도로 설정하면 모델이 새로운 주제나 단어를 도입하는 경향이 증가하여 출력의 다양성이 높아집니다.
# frequency_penalty: 이 값을 0.5~1.0 정도로 설정하면 특정 단어의 반복 사용을 억제하여 출력의 품질을 향상시킬 수 있습니다.

# 생성된 사업계획서 초안 출력
사업계획서_초안 = completion.choices[0].message.content
print("사업계획서 초안:\n", 사업계획서_초안)

사업계획서 초안:
 1. 제품 및 서비스 개발동기

수목의료 산업은 환경 보호와 도시 미관을 유지하는 중요한 역할을 하며, 나무의사의 진료기록 작성 및 관리, 처방 약품 발주 등의 작업은 이 산업의 핵심적인 부분입니다. 그러나 현재 나무병원에서는 이러한 작업들이 수작업으로 이루어지고 있어 효율성이 크게 떨어지고 있습니다. 특히, 진료기록을 체계적으로 작성하고 보관하는 시스템이 부재하여, 역사적인 데이터에 기반한 정밀한 진단 및 처방이 어려운 상황입니다. 이러한 문제는 환경 변화에 빠르게 대응하기 힘들게 하며, 산업 전반에 걸쳐 비효율성을 초래하고 있습니다. 이에 따라 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼은 나무의사와 조경업체가 보다 효율적으로 수목진료 정보를 공유하고, 처방 결과에 따라 필요한 약품과 자재를 적시에 발주할 수 있는 체계를 마련하는 것이 필요하게 되었습니다.

특히 이러한 시스템 부재는 원활한 물류 흐름을 저해하고 있으며, 결과적으로 고객 서비스 품질에도 부정적인 영향을 미치고 있습니다. 나무병원의 경우 신속하게 필요한 케미컬과 자재를 확보하지 못하면 진료가 지연되거나 품질 저하로 이어질 수 있습니다. 이로 인해 고객 불만이 증가하고 사업 신뢰도에도 타격을 입게 됩니다.

결론적으로 이러한 문제들을 해결하기 위해 우리의 O2O 플랫폼은 나무병원과 관련 업계 종사자들이 디지털 전환을 통해 업무 효율성을 극대화하고, 궁극적으로는 고객 만족도를 높이는 것을 목표로 하고 있습니다.

2. 제품 및 서비스의 목적(필요성)

제품 및 서비스의 핵심 목적은 수목관리 산업 내에서 정보 공유와 업무 프로세스의 효율화를 통해 전체 산업 생태계를 혁신하는 것입니다. 현재 나무병원 등이 처방전달시스템 없이 개별적으로 운영되는 상황에서 우리의 플랫폼은 이들을 하나로 연결하여 정보 격차를 해소하고자 합니다. 이를 통해 사용자들은 손쉽게 진료기록을 작성 및 관리할 수 있으며, 최적화된 처방전달시스템으로 필요한 케미컬과 자재를 효과적으로 관리할 수 있

### 문항 2개인 초안 3개 생성

In [ ]:
# 사업계획서 초안 생성을 위한 프롬프트 작성
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
"""

# OpenAI API를 사용하여 사업계획서 초안 3개 생성
drafts = []
for _ in range(3):
    completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content': prompt}],
                                                            temperature=0.7, # 창의성과 논리성의 균형(비즈니스 분석과 논리적 사고, 전략젝 제안이 필요하여)
                                                            presence_penalty=0.8, # 기존 내용에서 벗어난 새로운 개선점 추가를 위해
                                                            frequency_penalty=0.5,
                                                             max_tokens = 4000) # 다양한 표현을 주되, 중요 개념의 반복도 가능
    drafts.append(completion.choices[0].message.content)

# 생성된 사업계획서 초안 출력
for i, draft in enumerate(drafts, start=1):
    print(f"{i}번 초안:\n", draft)

1번 초안:
 ### 1. 문제 인식

#### 1-1. 제품 및 서비스 개발 동기
현재 나무의사와 관련된 진료 기록 작성, 보관, 그리고 처방 약품과 자재를 발주 및 관리하는 시스템이 전무한 상태입니다. 이러한 상황은 수목의료 산업 내에서 비효율적인 업무 처리와 정보의 분산을 초래하며, 전체적인 관리 효율성을 저하시킵니다. 특히, 나무병원에서는 환자의 건강 기록처럼 수목에 대한 진료 기록이 필수적이며, 이를 체계적으로 유지하는 것이 중요합니다. 하지만 현재는 대부분의 병원이 이러한 기록을 수작업으로 관리하고 있어 정보 누락이나 데이터 손실 가능성이 높습니다.

결론적으로, 이러한 비효율성을 극복하기 위해 OCS 기반의 시스템을 도입하여 진료 기록을 디지털화하고 체계적으로 관리할 필요가 있습니다. 이는 나무의사의 업무 효율성을 향상시키고 더 나은 서비스를 제공하기 위한 필수 요소로 작용할 것입니다.

#### 1-2. 제품 및 서비스의 목적(필요성)
나무병원의 주요 과제 중 하나는 농약 및 비료, 자재 및 소모품 등의 재고를 효율적으로 관리하는 것입니다. 그러나 현재 대부분의 나무병원은 이 모든 것을 수작업으로 처리하고 있으며, 이는 시간 소모뿐만 아니라 실수 발생 가능성을 높이는 원인이 됩니다. 이로 인해 실제 필요한 물품들이 적시에 공급되지 않거나 불필요한 재고가 쌓이는 문제가 발생하게 됩니다.

특히, 수목관리 산업 내에서 케미컬과 자재 관리는 매우 중요한 부분이며, 이를 효과적으로 운영하기 위해서는 디지털 플랫폼을 통한 자동화된 시스템이 필요합니다. 본 제품과 서비스를 통해 사용자는 간단한 인터페이스를 활용하여 실시간 재고 현황 파악 및 신속한 발주가 가능해질 것입니다.

결론적으로, 이 플랫폼은 단순히 효율성 증대뿐만 아니라 궁극적으로는 산업 전반에 걸친 활성화를 유도하게 될 것이며 다양한 이해관계자들 간에 원활한 정보를 공유함으로써 상호 협력을 증진시켜줄 것입니다.
2번 초안:
 1. **제품 및 서비스 개발동기**

나무병원과 조경업체는 수목의료에

### 문항 2개인 초안 3개를 점수화

In [16]:
def evaluate_draft(draft):
    """
    GPT-4o를 활용해 초안 평가 및 개선점 분석
    KOSME 평가 기준 반영: 문제 정의의 구체성과 설득력(40%), 차별화 요소 및 신뢰성 확보(30%), 시장 확장 가능성(20%), 논리적 전개(10%)
    """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성개 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "strengths": ["평가항목1 관련 강점", "평가항목2 관련 강점"],
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}

def select_best_draft(drafts):
    """초안 평가 및 최적 선택"""
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 선택된 초안의 핵심 강점(심사기준 연계)
    2. 잠재적 취약점 및 리스크 
    3. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }
    
    
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft']) 

★ 최종 선택: 2번 초안 (85/100)

▼ 합격률 높은 이유 분석 ▼
**분석 리포트**

1. **선택된 초안의 핵심 강점**

   - **문제 정의 및 필요성 설명**: 초안은 현재 나무병원과 조경업체가 직면한 비효율성을 명확하게 정의하고, 이러한 문제를 해결하기 위한 디지털 플랫폼의 필요성을 설득력 있게 제시하였습니다. 이는 심사기준에서 강조하는 '문제 인식 및 솔루션 제시' 측면에서 높은 평가를 받을 수 있습니다.
   
   - **차별화 요소**: 플랫폼이 제공하는 다양한 기능, 특히 병해충 확산 데이터 시각화를 통한 예방적 조치는 경쟁 제품들 사이에서 차별화를 이끌어내며 신뢰성을 확보합니다. 이는 '혁신성과 차별성' 기준과도 잘 부합합니다.

2. **잠재적 취약점 및 리스크**

   - **시장 확장 가능성 부족**: 글로벌 시장에서의 성공 가능성이 충분히 강조되지 않았습니다. 이는 국제 시장 진출을 목표로 할 경우 투자자나 이해관계자들에게 매력도를 낮출 수 있는 요소입니다.
   
   - **사용자 경험 부족**: 논리적 전개에서는 구체적인 사용자 경험 예시나 활용 시나리오가 부족하여 잠재 고객에게 서비스의 실제 유용함을 충분히 전달하지 못할 위험이 있습니다.

3. **KOSME 가이드라인 반영한 구체적 개선방안**

   - **글로벌 시장 전략 강화**: 글로벌 시장 진출을 위한 상세한 전략이나 계획을 기술하여, 어떻게 현지화 및 국제 협력을 통해 시장을 확대할 것인지에 대한 구체적인 방안을 제시해야 합니다. 이를 통해 잠재 투자자와 파트너에게 더 큰 신뢰를 줄 수 있습니다.
   
   - **구체적인 사용자 사례 추가**: 플랫폼 사용자의 실제 활용 사례나 기대 효과를 구체적인 예시로 설명함으로써 논리적 전개를 보강할 필요가 있습니다. 예를 들어, 특정 상황에서 어떻게 플랫폼이 문제 해결에 기여했는지를 스토리텔링 형식으로 보여주면 독자의 이해도를 높일 수 있습니다.

이러한 개선 사항들을 반영하면 초안은 더욱 강력한 경쟁력을

### 초안 3개 생성 후 평가하고 최종 초안 1개 선택 코드(1. 문제인식 부분)

In [ ]:
### 사업계획서 초안 생성 ###

import json

def generate_drafts(prompt, num_drafts=3):  # drafts=3으로 초안 3개 생성을 의미
    """ OpenAI API를 사용하여 사업계획서 초안 생성 """
    drafts = []    # 생성된 초안을 저장
    for _ in range(num_drafts):
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'system', 'content': prompt}],
            temperature=0.7,
            presence_penalty=0.8,
            frequency_penalty=0.5,
            max_tokens=4000
        )
        drafts.append(completion.choices[0].message.content)
    return drafts

### 생성된 drafts(초안)이 평가 항목으로 전달 ###

def evaluate_draft(draft):  # 3개의 초안을 가각 평가하여 점수를 부여하는 기능
    """ GPT-4o를 활용해 초안 평가 및 개선점 분석 """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "strengths": ["평가항목1 관련 강점", "평가항목2 관련 강점"],
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}


### 평가에 따른 합격률 높은 초안 선택 ###

def select_best_draft(drafts):  # 3개의 초안 평가 후 가장 높은 점수를 받은 초안 최종 선택
    """ 초안 평가 및 최적 선택 """
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 선택된 초안의 핵심 강점(심사기준 연계)
    2. 잠재적 취약점 및 리스크 
    3. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }

# 실행 코드
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스 개발동기와 서비스의 목적&필요성 부분만 작성해 주세요.
시장의 필요성과 법제화가 중요합니다.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

문제 인식 1: {item_info['제품 및 서비스 개발동기']}
문제 인식 2: {item_info['제품 및 서비스의 목적_필요성']}
"""

drafts = generate_drafts(prompt)
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft'])

★ 최종 선택: 2번 초안 (85/100)

▼ 합격률 높은 이유 분석 ▼
### 분석 리포트

#### 1. 선택된 초안의 핵심 강점

- **문제 정의의 구체성과 설득력**: 수목관리 산업에서 디지털화가 미흡하다는 점과 이에 따른 비효율성을 명확히 설명하여 문제를 잘 정의하였습니다. 이는 독자에게 현재 상황의 심각성을 이해시키고, 새로운 솔루션의 필요성을 강조하는 데 효과적입니다.

- **차별화 요소 및 신뢰성 확보**: 나무의사용 EMR 시스템을 개발하여 기존의 비효율적인 프로세스를 혁신하려는 시도가 돋보입니다. 특히 헬스케어 기술 전문기업 및 한양대학교 공과대학원과의 협력을 통해 프로젝트에 대한 신뢰성을 강화하고 있습니다. 이러한 협력은 제품이나 서비스 개발 과정에서 높은 기술력을 확보할 수 있음을 시사합니다.

#### 2. 잠재적 취약점 및 리스크

- **시장 확장 가능성 부족**: 제안된 플랫폼이 기존 시장 외에 새로운 고객층이나 해외 시장으로 어떻게 확장될 수 있을지에 대한 구체적인 설명이 부족합니다. 이는 장기적인 성장 가능성에 대한 의구심을 불러일으킬 수 있습니다.

- **논리적 전개의 부족**: 전체적인 논리 흐름은 이해가 되지만, 각 세부 항목 간 연결성이 조금 더 명확하면 좋겠습니다. 문제 정의에서부터 솔루션에 이르는 과정이 좀 더 자연스럽게 이어질 필요가 있습니다.

#### 3. KOSME 가이드라인 반영한 구체적 개선방안

- **구체적 개선방안1: 글로벌 진출 전략 마련**
  - 플랫폼이 국내 시장을 넘어 해외로 진출할 수 있는 전략을 마련하세요. 이를 위해 타겟 국가의 시장 조사를 실시하고, 현지 파트너와의 협업 가능성을 탐색해야 합니다.
  - 또한 다국어 지원 시스템 구축 및 현지 규제 준수 방안을 포함한 로드맵을 제시함으로써 글로벌 경쟁력을 강화할 수 있습니다.

- **구체적 개선방안2: 논리적 전개 강화**
  - 문제 정의, 해결책, 기대 효과 등 각 항목 간 연관성을 더욱 강조하여 서술하세요.
  - 예를 들

### 초안 3개 생성 후 평가하고 최종 초안 1개 선택 코드(2. 문제인식 부분)

In [18]:
### 사업계획서 초안 생성 ###

import json

def generate_drafts(prompt, num_drafts=3):  # drafts=3으로 초안 3개 생성을 의미
    """ OpenAI API를 사용하여 사업계획서 초안 생성 """
    drafts = []    # 생성된 초안을 저장
    for _ in range(num_drafts):
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'system', 'content': prompt}],
            temperature=0.7,
            presence_penalty=0.8,
            frequency_penalty=0.5,
            max_tokens=4000
        )
        drafts.append(completion.choices[0].message.content)
    return drafts

### 생성된 drafts(초안)이 평가 항목으로 전달 ###

def evaluate_draft(draft):  # 3개의 초안을 가각 평가하여 점수를 부여하는 기능
    """ GPT-4o를 활용해 초안 평가 및 개선점 분석 """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "strengths": ["평가항목1 관련 강점", "평가항목2 관련 강점"],
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}


### 평가에 따른 합격률 높은 초안 선택 ###

def select_best_draft(drafts):  # 3개의 초안 평가 후 가장 높은 점수를 받은 초안 최종 선택
    """ 초안 평가 및 최적 선택 """
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 선택된 초안의 핵심 강점(심사기준 연계)
    2. 잠재적 취약점 및 리스크 
    3. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }

# 실행 코드
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 제품 및 서비스의 개발 방안 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

실현 가능성 1: {item_info['제품 및 서비스의 개발 방안']}
"""

drafts = generate_drafts(prompt)
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft'])

★ 최종 선택: 2번 초안 (85/100)

▼ 합격률 높은 이유 분석 ▼
### 분석 리포트

#### 1. 선택된 초안의 핵심 강점

- **문제 정의의 구체성과 설득력**: 수목 관리에 특화된 EMR(전자 의료 기록) 시스템의 필요성을 명확히 설명하고, 이를 통해 헬스케어 전문기업인 시야인사이트와 한양대학교 공과대학원이 협업하여 시스템의 신뢰성을 높였다는 점이 강조되었습니다. 이는 사업계획서 심사기준에서 요구하는 문제 인식 및 해결방안 제시 항목을 효과적으로 충족합니다.

- **차별화 요소 및 신뢰성 확보**: 헬스케어 전문기업과의 협업으로 시스템의 신뢰성을 높이고, OCS(처방 전달 시스템) 개발 계획을 통한 차별화 전략이 돋보입니다. 이는 혁신성과 차별화를 중시하는 심사 기준에 부합하며, 시장 내 경쟁 우위를 점유할 가능성을 높입니다.

#### 2. 잠재적 취약점 및 리스크

- **시장 확장 가능성 부족**: 현재 초안은 시장 규모나 타겟 고객층에 대한 구체적인 분석이 부족합니다. 이는 사업 확장의 잠재력을 평가하기 어렵게 만듭니다. 수목 관리 EMR 시스템이 어느 정도의 시장 규모를 갖추고 있고, 성장 가능성이 얼마나 되는지에 대한 데이터가 없는 상태에서는 투자자에게 매력적이지 않을 수 있습니다.

- **논리적 전개의 미흡함**: 전체적으로 논리적인 흐름은 좋으나 문제 정의와 해결책 간의 직접적인 연결성이 다소 부족합니다. 즉, 문제로부터 도출된 해결책이 어떻게 구체적으로 작동하고 성공으로 이어질 것인지에 대한 명확한 설명이 필요합니다.

#### 3. KOSME 가이드라인 반영한 구체적 개선방안

- **구체적 개선방안1**: 시장 조사 데이터를 활용하여 수목관리 EMR 시스템에 대한 시장 크기와 성장 가능성을 제시하십시오. 예를 들어, 관련 산업 보고서나 통계 자료를 인용하여 이 시스템이 타겟으로 하는 고객층과 그들의 니즈를 정량적으로 설명하면 더 설득력이 있을 것입니다.

- **구체적 개선방안2**: 문제 정의에서 도출된 해결책이 어떻

In [63]:
### 사업계획서 초안 생성 ###

import json

def generate_drafts(prompt, num_drafts=3):  # drafts=3으로 초안 3개 생성을 의미
    """ OpenAI API를 사용하여 사업계획서 초안 생성 """
    drafts = []    # 생성된 초안을 저장
    for _ in range(num_drafts):
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'system', 'content': prompt}],
            temperature=0.7,
            presence_penalty=0.8,
            frequency_penalty=0.5,
            max_tokens=4000
        )
        drafts.append(completion.choices[0].message.content)
    return drafts

### 생성된 drafts(초안)이 평가 항목으로 전달 ###

def evaluate_draft(draft):  # 3개의 초안을 가각 평가하여 점수를 부여하는 기능
    """ GPT-4o를 활용해 초안 평가 및 개선점 분석 """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "strengths": ["평가항목1 관련 강점", "평가항목2 관련 강점"],
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}


### 평가에 따른 합격률 높은 초안 선택 ###

def select_best_draft(drafts):  # 3개의 초안 평가 후 가장 높은 점수를 받은 초안 최종 선택
    """ 초안 평가 및 최적 선택 """
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 선택된 초안의 핵심 강점(심사기준 연계)
    2. 잠재적 취약점 및 리스크 
    3. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }

# 실행 코드
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 자금소요 및 조달계획, 시장진입 및 성과창출 전략에서 내수시장 확보 방안(경쟁 및 판매가능성)과 해외시장 진출 방안(경쟁 및 판매가능성) 부분만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

성장 전략 1: {item_info['내수시장 확보 방안 경쟁 및 판매가능성']}
성장 전략 2: {item_info['해외시장 진출 방안 경쟁 및 판매가능성']}
성장 전략 3: {item_info['자금 조달 및 개발 방안']}
"""

drafts = generate_drafts(prompt)
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft'])

★ 최종 선택: 2번 초안 (84/100)

▼ 합격률 높은 이유 분석 ▼
**분석 리포트**

**1. 선택된 초안의 핵심 강점(심사기준 연계)**

이번 사업 계획서에서 두드러지는 핵심 강점은 문제 정의의 구체성과 설득력입니다. 본 사업이 다양한 분야에서 자금이 필요하다는 점과 이를 통해 초기 단계에서의 안정성을 확보하려는 목표가 명확히 제시되어 있습니다. 이는 심사 기준 중 '사업의 명확한 목표 설정 및 전략' 부분에서 높은 평가를 받을 수 있는 요소입니다.

또한, 해외시장 진출 전략에서는 시각화를 통한 차별화와 현지 규제 준수 계획을 상세히 언급하여 신뢰성을 높이고 있습니다. 이러한 차별화 요소는 경쟁업체와 차별되는 가치 제안을 만들어 낼 수 있으며, 이는 심사 기준 중 '경쟁 우위 확보 방안' 측면에서 긍정적으로 작용할 것입니다.

**2. 잠재적 취약점 및 리스크**

초안에서 나타난 잠재적 취약점으로는 내수시장과 해외시장의 구체적인 소비자 분석이나 수요 예측 데이터가 부족하다는 점이 있습니다. 시장 확장 가능성을 정확하게 파악하고 효과적으로 대응하기 위해서는 더 구체적인 데이터 기반의 접근이 필요합니다.

또한, 자금 조달 계획 부분에서는 정부지원금의 구체적인 항목 설명이 부족하여 논리적 전개의 완결성이 떨어질 수 있습니다. 이로 인해 투자자나 심사위원들이 자금 사용에 대한 명확한 그림을 그리기 어려울 수 있어 개선이 필요합니다.

**3. KOSME 가이드라인 반영한 구체적 개선방안**

구체적 개선방안1: 내수시장 확장을 위해 주 소비자층인 나무병원 및 조경업체에 대한 더 깊이 있는 분석을 실시해야 합니다. 이들의 니즈와 시장 트렌드를 파악하여 자료를 바탕으로 한 수요 예측 데이터를 포함시키면 시장 진입 전략의 타당성이 강화될 것입니다.

구체적 개선방안2: 자금 조달 계획에서는 정부지원금을 통해 충당할 비용 항목을 더욱 상세히 기술하십시오. 예를 들어, 시스템 개발비 중 어느 부분에 지원금을 사용할 것인지, 또는 어떤 인건비 항목에 활용할 

In [69]:
### 사업계획서 초안 생성 ###

import json

def generate_drafts(prompt, num_drafts=3):  # drafts=3으로 초안 3개 생성을 의미
    """ OpenAI API를 사용하여 사업계획서 초안 생성 """
    drafts = []    # 생성된 초안을 저장
    for _ in range(num_drafts):
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'system', 'content': prompt}],
            temperature=0.7,
            presence_penalty=0.8,
            frequency_penalty=0.5,
            max_tokens=4000
        )
        drafts.append(completion.choices[0].message.content)
    return drafts

### 생성된 drafts(초안)이 평가 항목으로 전달 ###

def evaluate_draft(draft):  # 3개의 초안을 가각 평가하여 점수를 부여하는 기능
    """ GPT-4o를 활용해 초안 평가 및 개선점 분석 """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 사업계획서 초안을 평가하세요:
    
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성 (20점)
    4. 논리적 전개 (10점)
    
    [초안 내용]
    {draft}
    
    [출력 형식]
    {{
      "score": 총점(0-100),
      "weaknesses": ["개선필요 항목1", "개선필요 항목2"],
      "improvements": ["구체적 개선방안1", "구체적 개선방안2"]
    }}
    """
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "strengths": [], "weaknesses": [], "improvements": []}


### 평가에 따른 합격률 높은 초안 선택 ###

def select_best_draft(drafts):  # 3개의 초안 평가 후 가장 높은 점수를 받은 초안 최종 선택
    """ 초안 평가 및 최적 선택 """
    evaluations = [evaluate_draft(draft) for draft in drafts]
    best_idx = max(enumerate(evaluations), key=lambda x: x[1]['score'])[0]
    
    # 최종 분석 리포트 생성
    analysis_prompt = f"""
    [초안 내용]
    {drafts[best_idx]}
    
    [평가 결과] 
    {evaluations[best_idx]}
    
    다음 구조로 분석 리포트 생성:
    1. 잠재적 취약점 및 리스크 
    2. KOSME 가이드라인 반영한 구체적 개선방안
    """
    
    analysis = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': analysis_prompt}],
        temperature=0.7,
        presence_penalty=0.8,
        frequency_penalty=0.5,
        max_tokens = 200
    ).choices[0].message.content
    
    return {
        'selected_index': best_idx+1,
        'score': evaluations[best_idx]['score'],
        'analysis': analysis,
        'selected_draft': drafts[best_idx]
    }

# 실행 코드
prompt = f"""
당신은 기존 창업가의 아이디어를 구체화하고 사업 가능성을 검증하는 사업계획서 작성 전문가입니다. 
{item_info}정보를 바탕으로 {kosme_template}템플릿 양식을 철저히 지켜 작성하며,
{kosme_pass}를 필수적으로 참고하세요.
아래 두 질문에 각각 15줄 이상 상세히 답변해주세요. 반드시 답변마다 별도의 문단을 생성하고, 간결함보다 완전성을 우선시합니다
이 사업계획서는 내 생계가 달린 아주 중요한 작업입니다.
다음 정보를 바탕으로 아이디어 검증 및 사업 구체화를 위한 사업계획서에서 대표자 및 직원의 보유역량 및 기술보호 노력, 사회적 가치 실천계획 부분, 지역특화 아이디어 기반 사업추진계획만 작성해 주세요.
관련된 정보를 더 찾아서 사업아이템을 구체화해줘도 좋습니다.
'특히' 또는 '결론적으로' 같은 단어가 있다면 줄바꿈을 해주세요.

팀 구성 1: {item_info['대표자 및 직원의 보유역량 및 기술보호 노력']}
팀 구성 2: {item_info['사회적 가치 실천계획']}
팀 구성 3: {item_info['지역특화 아이디어 기반 사업추진계획']}
"""

drafts = generate_drafts(prompt)
best_draft = select_best_draft(drafts)

print(f"★ 최종 선택: {best_draft['selected_index']}번 초안 ({best_draft['score']}/100)")
print("\n▼ 합격률 높은 이유 분석 ▼\n" + best_draft['analysis'])
print("\n▼ 선택된 초안 내용 ▼\n" + best_draft['selected_draft'])

★ 최종 선택: 3번 초안 (85/100)

▼ 합격률 높은 이유 분석 ▼
**분석 리포트**

**1. 잠재적 취약점 및 리스크**

이번 보고서에서는 사업 계획의 몇 가지 잠재적 취약점을 식별하였습니다:

- **문제 정의의 구체성과 설득력 부족**: 조경 및 수목관리 분야에서 해결하고자 하는 문제에 대한 구체적인 사례나 데이터를 제시하지 않았습니다. 이는 투자자나 이해관계자들이 사업의 필요성을 인식하는 데 어려움을 줄 수 있습니다.
  
- **시장 확장 가능성에 대한 구체적인 데이터 부족**: 국내외 시장 진입 전략이 명확하지 않으며, 목표 시장 규모나 경쟁 분석 등의 구체적인 데이터가 부족합니다. 이는 사업 확장 가능성을 평가하는 데 중요한 요소로 작용할 수 있습니다.

**2. KOSME 가이드라인 반영한 구체적 개선방안**

KOSME 가이드라인을 반영하여 다음과 같은

▼ 선택된 초안 내용 ▼
**4. 팀 구성**

**4-1. 대표자 및 직원의 보유역량 및 기술보호 노력**

대표자는 한양대학교 도시대학원에서 랜드스케이프어바니즘을 전공하였으며, 인천시설공단과 자이에스엔디에서 10년 동안 근무한 경험을 가지고 있습니다. 이러한 경력은 수목관리와 조경산업에 대한 깊은 이해와 전문성을 제공합니다. 특히, 다양한 프로젝트를 통해 실무적 역량을 쌓아왔고, 이는 OCS 기반 수목의료용 케미컬 및 자재 커머스 O2O 플랫폼 개발에 큰 자산이 될 것입니다.

기술 보호 측면에서는 현재까지 총 3건의 특허를 출원하였습니다. 이는 독창적인 기술과 아이디어가 경쟁사들로부터 보호받을 수 있도록 하는 중요한 요소입니다. 또한, 회사 내부적으로도 지식재산권 관리 시스템을 도입하여 지속적인 기술 혁신과 보호에 힘쓰고 있습니다. 이 외에도 신기술 개발 시 법률적 자문을 받아 지식재산권 관련 위협을 사전에 차단하고 있으며, 이를 통해 안정적인 사업 운영과 확장을 도모하고 있습니다.


**4-2. 사회적 가치 실천계획**

특히 양질의 일자리 창출은 우리의 핵심 목표 중 하

In [85]:
draft1 = """
현재 수목관리 산업은 디지털화가 미흡하여 여러 비효율성이 존재합니다. 특히, 나무의사의 진료기록을 작성하고 보관하는 체계적인 시스템이 없으며, 처방된 약품과 자재를 발주 및 관리하는 프로세스도 수작업으로 이루어지고 있습니다. 이러한 상황은 데이터를 통합적으로 관리할 수 없는 환경을 만들고 있으며, 결과적으로 나무병원과 조경업체들이 효과적으로 수목의료를 제공하는 데 큰 걸림돌로 작용하고 있습니다.
특히 OCS(처방전달시스템) 기반의 플랫폼이 존재하지 않아, 나무의사와 관련 업계 종사자들 간에 정확한 진료 정보와 자재 정보가 실시간으로 공유되지 못하고 있습니다. 이는 효율적인 업무 프로세스를 저해하며 오차 발생 가능성을 높입니다. 따라서 우리는 나무의사용 EMR 시스템을 헬스케어 기술 전문기업 시야인사이트 및 한양대학교 공과대학원과 공동개발하여 이러한 문제를 해소하고자 합니다.
결론적으로, 본 아이템은 기존의 비효율적인 시스템을 개선하기 위한 필수적이고 혁신적인 해결책으로서 개발되었으며, 이를 통해 관련 산업 전반에 걸쳐 업무 효율성을 극대화할 것입니다.
"""

In [86]:
draft2 = """
나무의사 자격 제도의 법제화(2018)로 수목의료 헬스케어 산업의 기반이 마련됨. 산림보호법의 개정을 통해 나무의사 자격제도가 만들어진 후 2019년 첫 시험을시작으로 현재까지 약 500명 이상의 나무의사가 배출되었으며, 나무의사 또는 수목치료기술자를 고용하는 나무병원 산업이 활성화되며 약 1,800여개의 전문 나무병원이 사업을 영위중임
국내 반려동물 시장과 비교하여 조경식재 및 관리시장이 유사하거나 큰 시장으로 성장하였음. 그럼에도 불구하고 나무의사 진료기록을 작성, 보관하거나 처방 약품과 자재를 발주 및 관리하는 시스템이 전무하였음. 따라서 의사나 수의사와 같이 나무의사도 전문적으로 진료기록을 작성하고 보관하는 솔루션 EMR의 필요성을 인식하였고 2022년 2월 시제품을 개발 완료하여 나무의사 및 수목관리 분야 전문가를 대상으로 CBT를 진행중임.
수목의료용 EMR과 연동한 OCS 기반의 약품 및 자재 O2O 플랫폼을 통한 수목의료 산업의 확대 및 체계화 실현. 당사에서 기개발한 EMR과 연동되는 OCS의 구축을 통해 EMR의 활용성 및 편의성을 보강할 뿐 아니라 나무병원업 외 케미컬 및 자재사 등과 연계되는 수목의료용 케미컬 및 자재 O2O 플랫폼 환경을 구축하고자 함.
해당 분야는 IT 연동성이 낮아 효율적 관리가 되지 못하는 분야임. 대기업에서 조경관리를 총괄하고 공공기관에서 각종 조경업무를 담당하며 경험한 수목진료 및 조경 산업의 pain-point를 현실적으로 반영한 솔루션의 필요성을 인식하여 개발을 시작함.
"""

In [ ]:
import json

def evaluate_drafts(draft1, draft2):
    """
    GPT-4o를 활용해 두 개의 초안을 비교 평가
    """
    evaluation_prompt = f"""
    [역할]
    당신은 KOSME 청년창업사관학교 심사위원입니다. 다음 기준으로 두 개의 사업계획서 초안을 비교하여 평가하세요.
    초안 내용은 '제품 및 서비스 개발동기'에 해당합니다. 

    [평가 기준]
    1. 문제 정의의 구체성과 설득력 (40점)
    2. 차별화 요소 및 신뢰성 확보 (30점)
    3. 시장 확장 가능성 (20점)
    4. 논리적 전개 (10점)

    [초안 1]
    {draft1}

    [초안 2]
    {draft2}

    [출력 형식 (반드시 JSON으로 출력하세요)]
    ```json
    {{
      "draft1_score": 초안1 총점 (0-100),
      "draft2_score": 초안2 총점 (0-100),
      "better_draft": "초안1" 또는 "초안2",
      "strengths_draft1": ["초안1의 강점1", "초안1의 강점2"],
      "strengths_draft2": ["초안2의 강점1", "초안2의 강점2"],
      "weaknesses_draft1": ["초안1의 개선 필요점1", "초안1의 개선 필요점2"],
      "weaknesses_draft2": ["초안2의 개선 필요점1", "초안2의 개선 필요점2"],
      "improvements_draft1": ["초안1 개선 방안1", "초안1 개선 방안2"],
      "improvements_draft2": ["초안2 개선 방안1", "초안2 개선 방안2"],
      "comparison_summary": "두 초안 비교 분석 요약"
    }}
    ```
    반드시 위 JSON 형식을 유지하여 출력하세요.
    """

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'system', 'content': evaluation_prompt}],
        temperature=0.7
    )

    content = response.choices[0].message.content.strip()

    # 응답이 JSON인지 확인 후 변환
    try:
        # JSON으로 변환
        if content.startswith("```json"):  # 코드 블록 제거
            content = content[7:-3].strip()  # "```json"과 "```" 제거
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(f"JSON 디코딩 실패: {e}")
        print(f"GPT-4o 응답 내용: {content}")
        return {
            "draft1_score": 0, "draft2_score": 0, "better_draft": "N/A",
            "strengths_draft1": [], "strengths_draft2": [],
            "weaknesses_draft1": [], "weaknesses_draft2": [],
            "improvements_draft1": [], "improvements_draft2": [],
            "comparison_summary": "평가 실패 (GPT-4o 응답 오류)"
        }

# ✅ 초안 비교 실행
comparison_result = evaluate_drafts(draft1, draft2)

# ✅ 결과 출력
print("\n★ 초안 비교 결과 ★")
print(f"📌 초안 1 점수: {comparison_result['draft1_score']}/100")
print(f"📌 초안 2 점수: {comparison_result['draft2_score']}/100")
print(f"✅ 더 나은 초안: {comparison_result['better_draft']}")
print("\n🔹 초안 1 강점:", ", ".join(comparison_result["strengths_draft1"]))
print("🔹 초안 2 강점:", ", ".join(comparison_result["strengths_draft2"]))
print("\n⚠️ 초안 1 약점:", ", ".join(comparison_result["weaknesses_draft1"]))
print("⚠️ 초안 2 약점:", ", ".join(comparison_result["weaknesses_draft2"]))
print("\n💡 초안 1 개선 방안:", ", ".join(comparison_result["improvements_draft1"]))
print("💡 초안 2 개선 방안:", ", ".join(comparison_result["improvements_draft2"]))
print("\n📌 🔥 비교 분석 요약 🔥")
print(comparison_result["comparison_summary"])




★ 초안 비교 결과 ★
📌 초안 1 점수: 83/100
📌 초안 2 점수: 90/100
✅ 더 나은 초안: 초안2

🔹 초안 1 강점: 문제 정의가 구체적이며 나무의사의 진료 기록 및 자재 관리의 비효율성을 지적함., 헬스케어 기술 전문기업 및 대학과의 공동 개발로 신뢰성 있는 협업을 강조함.
🔹 초안 2 강점: 나무의사 자격 제도의 법제화 및 시장 성장 배경을 설명하여 문제의 구체성을 높임., 이미 시제품 개발 완료 및 CBT 진행 중임을 언급하여 신뢰성을 강화함.

⚠️ 초안 1 약점: 시장 확장 가능성에 대한 구체적인 설명이 부족함., 논리적 전개에서 문제 해결을 위한 프로세스가 다소 간결함.
⚠️ 초안 2 약점: 지나치게 많은 정보로 인해 초점이 분산될 가능성이 있음., 차별화 요소가 명확히 드러나지 않음.

💡 초안 1 개선 방안: 시장 확장 가능성을 구체적으로 설명하여 설득력을 높일 필요가 있음., 문제 해결을 위한 단계별 접근 방식을 상세히 설명하여 논리적 전개를 강화할 필요가 있음.
💡 초안 2 개선 방안: 차별화되는 기술적 요소를 명확히 제시하여 경쟁력을 강화할 필요가 있음., 정보의 초점을 명확히 하여 핵심을 부각시키는 것이 필요함.

📌 🔥 비교 분석 요약 🔥
초안1은 문제 정의와 신뢰성 있는 협업을 강조하였으나 시장 확장 가능성에 대한 구체적 설명이 부족하다. 초안2는 시장 배경과 신뢰성을 잘 나타내었으며, 이미 시제품 개발 및 CBT가 진행 중임을 언급하여 실제적 신뢰성을 강화하였다. 그러나 초안2는 정보의 초점이 다소 분산되어 있으며 차별화 요소가 명확하지 않을 수 있다. 전반적으로 초안2가 더 완성도 높은 계획으로 평가된다.


### gpt-3.5-turbo 고도화하여 4o모델 수준까지 끌어올리기
##### 1. rag 구축 
- 1-1. 사업계획서 작성과 관련된 데이터 수집(사업계획서 샘플, 평가기준, 가이드라인, 청창사 성공 사례, 사업계획서 실수하는 항목 정리 등 )
- 1-2. json, csv 또는 vectorDB로 저장- vectorDB를 사용할 계획(GPT-3.5가 입력 받을 때 관련 데이터를 검색하여 자동으로 제공)

##### 2. GPT-3.5와 데이터베이스 연결(RAG 적용)
- 2-1
- rag를 먼저 구축하면, 최신 데이터를 검색하여 gpt-3.5가 즉각적으로 활용하고 fine-tuning으로 rag를 보완해주면 좋다. 그리고 gpt-4o로 검토 및 수정

In [ ]:
pip install faiss-cpu langchain openai pypdf pandas


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path

# ✅ 특정 폴더 경로 설정
folder_path = Path(r"C:/Users/MSI/Desktop/final project/kosme_file")

# ✅ 해당 폴더 내의 PDF 파일만 가져오기
pdf_files = list(folder_path.glob("*.pdf"))

# ✅ 가져온 파일 목록 확인
print("📂 가져온 PDF 파일 리스트:", pdf_files)


📂 가져온 PDF 파일 리스트: [WindowsPath('C:/Users/MSI/Desktop/final project/kosme_file/2022년+창업성공패키지+청년창업사관학교+신청서_.pdf'), WindowsPath('C:/Users/MSI/Desktop/final project/kosme_file/사업계획서 100번 써본 대표가 알려주는 사업계획서 합격 노하우.pdf')]


In [ ]:
pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# NLTK의 punkt tokenizer 다운로드 (처음 한 번만 실행)
nltk.download('punkt')

In [ ]:
pip install langchain-chroma

In [ ]:
# Langchain에서 지원해주는 Chroma 클래스 임포트
from langchain_chroma import Chroma

# Langchain과 연동된 openai에서 지원하는 pre-trained Embedding 모델 활용
from langchain_openai import OpenAIEmbeddings

In [ ]:
# 임베딩 설정
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                                api_key = MY_API_KEY
                                )

In [ ]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma

# PDF 파일이 있는 디렉토리 경로
pdf_directory = 'C:/Users/MSI/Desktop/final project/kosme_file'
my_directory = 'VectorStores/'

# 청킹 설정
chunk_size = 200  # 분할된 텍스트의 최대 길이
chunk_overlap = 5  # 텍스트 조각 간의 겹치는 부분의 길이

# RecursiveCharacterTextSplitter 초기화
csv_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 디렉토리 내의 모든 PDF 파일 읽기
documents = []
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):  # PDF 파일만 선택
        pdf_file_path = os.path.join(pdf_directory, filename)

        # PDF 파일 읽기
        with open(pdf_file_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            
            # 모든 페이지의 텍스트를 하나의 문자열로 결합
            for page in reader.pages:
                text += page.extract_text() + "\n"

        # 텍스트를 청킹
        chunks = csv_splitter.split_text(text)

        # 청킹된 텍스트를 Document 객체로 변환
        for chunk in chunks:
            documents.append(Document(page_content=chunk, metadata={"source": filename}))

print("청크 개수 :", len(chunks))

# Chroma DB 생성
vectordb = Chroma.from_documents(documents=documents, embedding=my_embedding, persist_directory=my_directory)

# 결과 출력
print("Chroma DB가 성공적으로 생성되었습니다.")


청크 개수 : 81
Chroma DB가 성공적으로 생성되었습니다.


In [ ]:
vectordb._collection.count()

81

In [ ]:
# 모든 데이터 삭제
vectordb.delete_collection()

In [ ]:
question = "사업계획서 잘 쓰는 방법은?"

# similarity_search : 유사도 검색 실행 함수
 # k : 반환 받고자하는 document 객체 수 지정(청크 수)
 # (k값이 너무 크면 연관성이 떨어지는 문서까지 광범위하게 검색되어 정확성이 낮아질 수 있음)
similar_docs = vectordb.similarity_search(question, k=3)

print("객체 수 :", len(similar_docs))
print(similar_docs)

객체 수 : 3
[Document(metadata={'source': '사업계획서 100번 써본 대표가 알려주는 사업계획서 합격 노하우.pdf'}, page_content='사 업 계 획 서  작 성 은 ,  최 대 한  구 체 적 으 로 !'), Document(metadata={'source': '사업계획서 100번 써본 대표가 알려주는 사업계획서 합격 노하우.pdf'}, page_content='김 정 현신 해 람사업계획서 100번 써본 대표가 알려주는 사업계획서 합격 노하우\n\n정 부 지 원 사 업  소 개\n\n정 부 지 원 사 업  지 원  시 ,\n꼭  확 인 해 야  하 는  홈 페 이 지  및  채 널\nK 스 타 트 업\n스 타 트 업  플 러 스\n기 업 마 당\n판 판 대 로\nR I P C  지 역 지 식 재 산 센 터'), Document(metadata={'source': '사업계획서 100번 써본 대표가 알려주는 사업계획서 합격 노하우.pdf'}, page_content='모 바 일  게 임  창 업 의  경 우  사 업 계 획 서 를  어 떤  식 으 로  작 성 하 면  좋 을 까 요 ?모 바 일  게 임 의  경 우 ,  \n사 업 계 획 서 를  쓰 실  때 는   합 니 다 .')]
